In [35]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as scs

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline


In [113]:
df = pd.read_csv('datawithnewcols.csv',index_col = 0)

In [82]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,long,sqft_living15,sqft_lot15,Age_of_House,Years_last_renovation,basement,distance_from_city,distance_from_airport,sqft_living_comparison,sqft_lot_comparison
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,True,0.0,...,-122.257,1340,5650,65,65,False,12.251612,8.232375,0.880597,1.000000
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,False,0.0,...,-122.319,1690,7639,69,29,True,12.613856,30.859494,1.520710,0.948030
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,False,0.0,...,-122.233,2720,8062,87,87,False,16.333282,33.112379,0.283088,1.240387
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,False,0.0,...,-122.393,1360,5000,55,55,True,10.617501,10.988905,1.441176,1.000000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,False,0.0,...,-122.045,1800,7503,33,33,False,21.759524,27.230991,0.933333,1.076903


In [83]:
categoricals = ['floors','view','condition','grade','waterfront','basement']

In [112]:
numericals = ['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'sqft_living15',
'zipcode',
 'sqft_lot15',
 'Age_of_House',
 'Years_last_renovation',
 'distance_from_city',
 'distance_from_airport']

# Transform Numericals

In [130]:
# data_log = pd.DataFrame([])

# for name in numericals:
#     title = name + '_log'
#     data_log[title] = df[name].map(lambda x: np.log(x))    
# data_log

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

data_log_norm = df[numericals].apply(normalize)
data_log_norm

,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_living15,zipcode,sqft_lot15,Age_of_House,Years_last_renovation,distance_from_city,distance_from_airport
0,-0.402894,-1.451039,-0.980629,-0.228177,-0.943654,1.869602,-0.260621,0.544665,0.619965,-0.587518,-1.389853
1,-0.402894,0.174482,0.533357,-0.189735,-0.432877,0.879190,-0.187695,0.680835,-0.623756,-0.553744,0.960540
2,-1.482459,-1.451039,-1.427201,-0.123137,1.070267,-0.933451,-0.172186,1.293596,1.380017,-0.206959,1.194558
3,0.676671,1.149794,-0.131054,-0.243873,-0.914467,1.084747,-0.284453,0.204243,0.274487,-0.739876,-1.103518
4,-0.402894,-0.150622,-0.436030,-0.169499,-0.272347,-0.073848,-0.192682,-0.544688,-0.485565,0.298963,0.583629
...,...,...,...,...,...,...,...,...,...,...,...
21592,-0.402894,0.499586,-0.599410,-0.337298,-0.666375,0.468075,-0.412448,-1.293618,-1.245617,-0.780981,0.727336
21593,0.676671,0.499586,0.250165,-0.224241,-0.228566,1.271617,-0.203791,-1.463829,-1.418356,-0.704257,-1.338069
21594,-1.482459,-1.776143,-1.154901,-0.332010,-1.410650,1.234243,-0.394189,-1.293618,-1.245617,-1.440441,-0.503541
21595,-0.402894,0.499586,-0.523166,-0.306945,-0.841498,-0.952138,-0.420587,-1.123406,-1.072878,0.280613,-0.148423


# Transform Categoricals

In [53]:
df[categoricals]

,floors,view,condition,grade,waterfront,basement
0,1.0,0.0,3,7,True,False
1,2.0,0.0,3,7,False,True
2,1.0,0.0,3,6,False,False
3,1.0,0.0,5,7,False,True
4,1.0,0.0,3,8,False,False
...,...,...,...,...,...,...
21592,3.0,0.0,3,8,False,False
21593,2.0,0.0,3,8,False,False
21594,2.0,0.0,3,7,False,False
21595,2.0,0.0,3,8,True,False


In [59]:
dfnew = pd.DataFrame([])
for name in categoricals:
    dfnew[name] = pd.get_dummies(df[name], prefix=name, drop_first=True)
    dfnew[name] = dummies
    

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [86]:
categoricals

['floors', 'view', 'condition', 'grade', 'waterfront', 'basement']

In [87]:
floor_dummies = pd.get_dummies(df['floors'], prefix='floor', drop_first=True)
view_dummies = pd.get_dummies(df['view'], prefix='view', drop_first=True)
cond_dummies = pd.get_dummies(df['condition'], prefix='cond', drop_first=True)
grade_dummies = pd.get_dummies(df['grade'], prefix='grade', drop_first=True)
water_dummies = pd.get_dummies(df['waterfront'], prefix='water', drop_first=True)
base_dummies = pd.get_dummies(df['basement'], prefix='base', drop_first=True)

In [92]:
dfnew  = pd.concat([data_log_norm,floor_dummies,view_dummies,cond_dummies,grade_dummies,water_dummies,base_dummies],axis = 1)

In [115]:
data_log_norm.head()

,bedrooms_log,bathrooms_log,sqft_living_log,sqft_lot_log,sqft_living15_log,zipcode_log,sqft_lot15_log,Age_of_House_log,Years_last_renovation_log,distance_from_city_log,distance_from_airport_log
0,-0.279718,-1.726509,-1.125556,-0.388430,-1.035429,1.869128,-0.395450,0.687861,0.743973,-0.337593,-1.498558
1,-0.279718,0.339653,0.709446,-0.113241,-0.326781,0.879349,-0.024287,0.767756,-0.324649,-0.293581,0.867495
2,-1.711611,-1.726509,-2.131893,0.244475,1.126504,-0.933519,0.042036,1.077870,1.129959,0.096727,0.993666
3,0.736226,1.072635,0.070674,-0.523914,-0.990187,1.084819,-0.545850,0.464369,0.522787,-0.553816,-0.981411
4,-0.279718,0.039555,-0.292725,0.008139,-0.134218,-0.073585,-0.046394,-0.219034,-0.153568,0.529987,0.643510


In [131]:
data_log_norm['price'] = df['price']

In [94]:
dfnew['price'] = df.price

In [90]:
dfnew.head()

,bedrooms_log,bathrooms_log,sqft_living_log,sqft_lot_log,sqft_living15_log,lat_log,long_log,sqft_lot15_log,Age_of_House_log,Years_last_renovation_log,...,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13,water_True,base_True
0,-0.279718,-1.726509,-1.125556,-0.388430,-1.035429,-0.351359,NaN,-0.395450,0.687861,0.743973,...,0,1,0,0,0,0,0,0,1,0
1,-0.279718,0.339653,0.709446,-0.113241,-0.326781,1.160018,NaN,-0.024287,0.767756,-0.324649,...,0,1,0,0,0,0,0,0,0,1
2,-1.711611,-1.726509,-2.131893,0.244475,1.126504,1.281474,NaN,0.042036,1.077870,1.129959,...,1,0,0,0,0,0,0,0,0,0
3,0.736226,1.072635,0.070674,-0.523914,-0.990187,-0.282056,NaN,-0.545850,0.464369,0.522787,...,0,1,0,0,0,0,0,0,0,1
4,-0.279718,0.039555,-0.292725,0.008139,-0.134218,0.410205,NaN,-0.046394,-0.219034,-0.153568,...,0,0,1,0,0,0,0,0,0,0


In [122]:
modeldata = df[numericals]

In [125]:
modeldata.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_living15,zipcode,sqft_lot15,Age_of_House,Years_last_renovation,distance_from_city,distance_from_airport,price
0,3,1.00,1180,5650,1340,98178,5650,65,65,12.251612,8.232375,221900.0
1,3,2.25,2570,7242,1690,98125,7639,69,29,12.613856,30.859494,538000.0
2,2,1.00,770,10000,2720,98028,8062,87,87,16.333282,33.112379,180000.0
3,4,3.00,1960,5000,1360,98136,5000,55,55,10.617501,10.988905,604000.0
4,3,2.00,1680,8080,1800,98074,7503,33,33,21.759524,27.230991,510000.0


In [124]:
modeldata['price']=df['price']

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [117]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [132]:
X = data_log_norm.drop('price',axis = 1)
y= data_log_norm['price']

In [126]:
X = modeldata.drop('price',axis = 1)
y = modeldata['price']

In [127]:
y

0        221900.0
1        538000.0
2        180000.0
3        604000.0
4        510000.0
           ...   
21592    360000.0
21593    400000.0
21594    402101.0
21595    400000.0
21596    325000.0
Name: price, Length: 21597, dtype: float64

In [133]:
model = sm.OLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.212
Model:                            OLS   Adj. R-squared (uncentered):              0.211
Method:                 Least Squares   F-statistic:                              527.2
Date:                Mon, 01 Jun 2020   Prob (F-statistic):                        0.00
Time:                        12:06:25   Log-Likelihood:                     -3.1726e+05
No. Observations:               21597   AIC:                                  6.345e+05
Df Residuals:                   21586   BIC:                                  6.346e+05
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
bedrooms              -4.987e+04   4994.623     -9.984      0.000   -5.97e+04   -4.01e+04
bathrooms              3.285e+04   6976.320      4.709      0.000    1.92e+04    4.65e+04
sqft_living            2.259e+05   8178.965     27.619      0.000     2.1e+05    2.42e+05
sqft_lot                1.26e+04   5715.794      2.204      0.028    1394.242    2.38e+04
sqft_living15          5.893e+04   6238.208      9.446      0.000    4.67e+04    7.12e+04
zipcode               -4.122e+04   4834.345     -8.527      0.000   -5.07e+04   -3.17e+04
sqft_lot15            -2920.2827   5784.656     -0.505      0.614   -1.43e+04    8418.070
Age_of_House           7.219e+04   1.07e+04      6.736      0.000    5.12e+04    9.32e+04
Years_last_renovation -3.054e+04   1.07e+04     -2.852      0.004   -5.15e+04   -9549.139
distance_from_city    -1.454e+05   5080.453    -28.628      0.000   -1.55e+05   -1.35e+05
distance_from_airport  4.608e+04   4080.768     11.292      0.000    3.81e+04    5.41e+04
==============================================================================
Omnibus:                    19043.407   Durbin-Watson:                   0.264
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1679514.503
Skew:                           3.846   Prob(JB):                         0.00
Kurtosis:                      45.511   Cond. No.                         7.30
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [134]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 10)
selector = selector.fit(X, y.values.ravel()) # convert y to 1d np array to prevent DataConversionWarning
selector.support_ 

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True])

In [135]:
selected_columns = X.columns[selector.support_ ]
linreg.fit(X[selected_columns],y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [136]:
selected_columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_living15',
       'zipcode', 'Age_of_House', 'Years_last_renovation',
       'distance_from_city', 'distance_from_airport'],
      dtype='object')